In [ ]:
import pandas as pd
import numpy as np
import time

df = pd.read_excel('Dados de 2021 - Coletados em 17-06-2024 - REV02.xlsx')
df

In [1]:
import pandas as pd
import numpy as np
import time

# Filtrando Data Frame

In [ ]:
indices_colunas = list(range(16))

df_filtrado = df.iloc[:, indices_colunas]

df_filtrado

In [ ]:
df_filtrado.set_index(df_filtrado.columns[0], inplace=True)
df_filtrado

In [ ]:
df_filtrado = df_filtrado.drop(df_filtrado.columns[0], axis=1)
df_filtrado

In [ ]:
col_perdas = df_filtrado.columns[2]   

new_column_order = [col_perdas] + [col for i, col in enumerate(df_filtrado.columns) if i != 2]

df_filtrado = df_filtrado[new_column_order]

df_filtrado

In [ ]:
df_filtrado.isnull().sum()

In [ ]:
df_filtrado.describe()

# Data Frame com valores numéricos

In [ ]:
df_filtrado_numerico = df_filtrado.select_dtypes(include=[int, float])
df_filtrado_numerico

# <font color='red' style='font-size: 30px;'>Removendo variaveis fracas</font>
<hr style='border: 2px solid red;'>

In [ ]:
indices_para_remover = [2, 3, 4, 10]

df_filtrado_numerico = df_filtrado_numerico.drop(df_filtrado_numerico.columns[indices_para_remover], axis=1)

df_filtrado_numerico

# <font color='red' style='font-size: 30px;'>Separando dados em Treino e Teste</font>
<hr style='border: 2px solid red;'>

In [ ]:
y = df_filtrado_numerico['IN049_AE - Índice de perdas na distribuição']
y

In [ ]:
X = df_filtrado_numerico.drop('IN049_AE - Índice de perdas na distribuição', axis=1)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# <font color='red' style='font-size: 30px;'>Aplicando o MLJAR</font>
<hr style='border: 2px solid red;'>

In [ ]:
from supervised.automl import AutoML


automl = AutoML(
    mode="Compete",
    total_time_limit=9*60*60,     # 8 horas 
    hill_climbing_steps=3,        
    algorithms=["CatBoost"],
    eval_metric="r2",
    train_ensemble=False,
    explain_level=2,
    verbose=1,

    # Desativa transformações automáticas que alteram o pipeline
    golden_features=False,
    features_selection=False,
    kmeans_features=False,
    mix_encoding=False,           
    stack_models = False,

    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 5,
        "shuffle": True,
        "random_seed": 123
    }
)


In [ ]:
automl.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)
print("R² no conjunto de teste:", r2_score(y_test, y_pred))

# <font color='green' style='font-size: 20px;'>AutoML 4 : best model: 43_CatBoost_GoldenFeatures</font>

R² no conjunto de treino: 0.880779 <br>
R² no conjunto de teste: 0.8781906990180091

# <font color='green' style='font-size: 20px;'>AutoML 5 : best model: 43_LightGBM_GoldenFeatures_SelectedFeatures
</font>

R² no conjunto de treino: 0.894628  
R² no conjunto de teste: 0.8850684919051792

# <font color='green' style='font-size: 20px;'>AutoML 12 : best model: 3_Optuna_CatBoost</font>
R² no conjunto de treino: 0.746302  
R² no conjunto de teste: 0.7525573554681504

info:
{'additional': {'max_rounds': 10000,
  'early_stopping_rounds': 50,
  'max_rows_limit': None,
  'max_cols_limit': None},
 'preprocessing': {'columns_preprocessing': {},
  'target_preprocessing': ['na_exclude', 'scale_normal'],
  'ml_task': 'regression'},
 'validation_strategy': {'validation_type': 'kfold',
  'k_folds': 5,
  'shuffle': True,
  'random_seed': 123,
  'X_path': 'AutoML_12\\X.data',
  'y_path': 'AutoML_12\\y.data',
  'results_path': 'AutoML_12'},
 'learner': {'model_type': 'CatBoost',
  'ml_task': 'regression',
  'n_jobs': -1,
  'learning_rate': 0.1,
  'depth': 5,
  'rsm': 0.9246918654641184,
  'loss_function': 'RMSE',
  'seed': 1234,
  'eval_metric': 'R2',
  'l2_leaf_reg': 1.8855050729313414,
  'random_strength': 0.23602519218824375,
  'min_data_in_leaf': 19,
  'num_boost_round': 1000,
  'early_stopping_rounds': 50,
  'explain_level': 2},
 'automl_random_state': 1234,
 'ml_task': 'regression',
 'explain_level': 2,
 'name': '3_Optuna_CatBoost',
 'status': 'trained',
 'final_loss': -0.746301524720365,
 'train_time': 19.184600114822388,
 'data_type': 'original',
 'optuna_time_budget': 3600,
 'optuna_init_params': {'original_LightGBM': {'learning_rate': 0.025,
   'num_leaves': 32,
   'lambda_l1': 2.021363084622114e-07,
   'lambda_l2': 0.000677083207312605,
   'feature_fraction': 0.9675383295827803,
   'bagging_fraction': 0.7545241053102847,
   'bagging_freq': 1,
   'min_data_in_leaf': 1,
   'extra_trees': False,
   'metric': 'custom',
   'custom_eval_metric_name': 'r2',
   'num_boost_round': 1000,
   'early_stopping_rounds': 50,
   'cat_feature': [],
   'feature_pre_filter': False,
   'seed': 1234},
  'original_Xgboost': {'eta': 0.1,
   'max_depth': 4,
   'lambda': 1.5095984364197903,
   'alpha': 2.3216966098992468e-05,
   'colsample_bytree': 0.9306558431607892,
   'subsample': 0.8980114742444467,
   'min_child_weight': 1,
   'objective': 'reg:squarederror',
   'eval_metric': 'r2',
   'max_rounds': 1000,
   'early_stopping_rounds': 50,
   'seed': 1234},
  'original_CatBoost': {'learning_rate': 0.1,
   'depth': 5,
   'l2_leaf_reg': 1.8855050729313414,
   'random_strength': 0.23602519218824375,
   'rsm': 0.9246918654641184,
   'min_data_in_leaf': 19,
   'eval_metric': 'R2',
   'num_boost_round': 1000,
   'early_stopping_rounds': 50,
   'seed': 1234}},
 'optuna_verbose': True,
 'max_time_for_learner': 3600}
 
 
# <font color='green' style='font-size: 20px;'>AutoML 13 : best model: 3_Optuna_CatBoost</font>

(usado atualmente) melhor com 0.75 no teste manual

In [ ]:
# dir(automl)

In [ ]:
# dir(automl._best_model)

In [ ]:
automl._best_model.params

In [ ]:
from supervised.automl import AutoML

dir(AutoML)

In [ ]:
# automl.get_leaderboard()
